In [21]:
import pandas as pd
from anndata import AnnData
from scipy.sparse import csr_matrix
import scanpy as sc
import numpy as np

In [4]:
RNA = pd.read_table('AML1026_1_count_matrix_f100.txt.gz',index_col=0)

In [5]:
RNA_bygenes = RNA.T

In [6]:
RNA_bygenes

barcode,AACAACCTAAAGACTGGT,AACAACCTAAAGTCATGA,AACAACCTACAGTATGAT,AACAACCTACATCGAAGC,AACAACCTACCACGAAGG,AACAACCTACTCGACTGC,AACAACCTAGCAGGAGAA,AACAACCTAGCCGAGTCT,AACAACCTAGCGAAGCTC,AACAACCTATAGAGCCAG,...,TTGTCAACCTGTAACAGT,TTGTCAACCTTGATCGTA,TTGTTAGAGACGCTAGTG,TTGTTAGAGATTCGAAGA,TTGTTAGAGCAACGATCG,TTGTTAGAGCCGCTATCC,TTGTTAGAGGTCCTAGTT,TTGTTAGAGTACTAGTAG,TTGTTAGAGTAGCGCAGC,TTGTTAGAGTGAACGTCA
ABCE1_1,2,3,4,0,7,0,0,0,3,0,...,2,2,1,3,0,0,2,0,12,3
ABCE1_2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ABHD17A_1,1,0,1,0,0,0,0,0,0,0,...,2,1,2,1,0,0,0,0,4,1
ABHD17A_2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ABHD17C_1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZP3_2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,0,1,1,1,0
ZRSR2_1,1,0,0,0,0,0,0,0,0,0,...,1,1,1,1,0,0,0,0,1,0
ZRSR2_2,0,1,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
ZWINT_1,0,5,1,0,1,1,0,0,3,2,...,1,2,0,3,0,0,1,1,8,1


In [7]:
RNA_bygenes['genes'] = RNA_bygenes.index

In [8]:
RNA_bygenes['genes'] = RNA_bygenes.genes.apply(lambda x: x.split('_')[0])

In [9]:
RNA_bygenes_grouped = RNA_bygenes.groupby('genes').mean()

In [11]:
RNA_bygenes_grouped = AnnData(RNA_bygenes_grouped.T)

/home/meb521/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  """Entry point for launching an IPython kernel.


In [12]:
RNA_bygenes_grouped.obs.index = RNA_bygenes_grouped.obs.index + '-1'

In [14]:
RNA_bygenes_grouped.X = csr_matrix(RNA_bygenes_grouped.X)

In [15]:
adata = RNA_bygenes_grouped.copy()

In [16]:
adata

AnnData object with n_obs × n_vars = 17267 × 1201

In [19]:
sc.pp.filter_genes(adata, min_counts=1)
sc.pp.filter_cells(adata, min_genes=30)
sc.pp.calculate_qc_metrics(adata, percent_top=None, log1p=None, inplace=True)

In [22]:
adata.obs['log10_genes'] = np.log10(adata.obs.n_genes_by_counts)
adata.obs['log10_counts'] = np.log10(adata.obs.total_counts)

In [23]:
adata

AnnData object with n_obs × n_vars = 17237 × 1201
    obs: 'n_genes', 'n_genes_by_counts', 'total_counts', 'log10_genes', 'log10_counts'
    var: 'n_counts', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'

In [48]:
adata.write_h5ad('AML1026_2_mean.h5ad', compression='gzip')